In [18]:
import numpy as np
import pandas as pd
import re

### Load the data

In [19]:
orders = pd.read_csv("../../data/instacart/orders.csv")

orders_products_1 = pd.read_csv("../../data/instacart/order_products__prior.csv")
orders_products_2 = pd.read_csv("../../data/instacart/order_products__train.csv")
orders_products = pd.concat([orders_products_1, orders_products_2],ignore_index=True)

products = pd.read_csv("../../data/instacart/products_cropped.csv")

### Do we want only one purchase per customer?

In [20]:
one_purchase = False

### Create list of orders for each user and list of products for each order

In [21]:
user_to_order = orders.groupby('user_id')['order_id'].apply(list)

In [22]:
order_to_products = orders_products.groupby('order_id')['product_id'].apply(list)

### We will find a list of products bought (ever in any purchase) by users

In [23]:
user_to_products_id = {}

for user_id in user_to_order.index:
    orders = user_to_order[user_id]
        
    user_to_products_id[user_id] = []  
        
    for order in orders:
        if order in order_to_products.keys():
            user_to_products_id[user_id] += order_to_products[order]
            
        if one_purchase:
            break

In [24]:
user_to_products_text = {}

for user_id in user_to_order.index:
    prod_ids = user_to_products_id[user_id]
    names = products[products["product_id"].isin(prod_ids)].product_name.tolist()
    names = " ".join(str(v) for v in names)
    names = re.sub("[^\w]", " ",  names).split()
    names = np.unique(names)
    
    user_to_products_text[user_id] = names

In [25]:
import pickle

if one_purchase:
    with open('../../data/instacart/user_to_products_one_purchase_id.p', 'wb') as fp:
        pickle.dump(user_to_products_id, fp, protocol=pickle.HIGHEST_PROTOCOL)

    with open('../../data/instacart/user_to_products_one_purchase_text.p', 'wb') as fp:
        pickle.dump(user_to_products_text, fp, protocol=pickle.HIGHEST_PROTOCOL)    
else:
    with open('../../data/instacart/user_to_products_one_id.p', 'wb') as fp:
        pickle.dump(user_to_products_id, fp, protocol=pickle.HIGHEST_PROTOCOL)

    with open('../../data/instacart/user_to_products_text.p', 'wb') as fp:
        pickle.dump(user_to_products_text, fp, protocol=pickle.HIGHEST_PROTOCOL)